In [0]:
from pyspark.sql.functions import current_date

# Load source table
player_activity_df = spark.table("dev_catalog.silver.player_activity")

# Load dimension tables
dim_game_df = spark.table("dev_catalog.gold.dim_game").selectExpr("sk_id as sk_id_game", "GameID")
dim_activity_type_df = spark.table("dev_catalog.gold.dim_activity_type").selectExpr("sk_id as sk_id_activity_type", "Activity_type")
dim_play_mode_df = spark.table("dev_catalog.gold.dim_play_mode").selectExpr("sk_id as sk_id_play_mode", "PlayMode")

# Join dimension tables with the source table
fact_player_activity_df = player_activity_df \
    .join(dim_game_df, player_activity_df.GameID == dim_game_df.GameID, "inner") \
    .join(dim_activity_type_df, player_activity_df.Activity_type == dim_activity_type_df.Activity_type, "inner") \
    .join(dim_play_mode_df, player_activity_df.PlayMode == dim_play_mode_df.PlayMode, "inner") \
    .select(player_activity_df["PlayerID"], player_activity_df["SessionID"], player_activity_df["Level"], player_activity_df["ExperiencePoints"], player_activity_df["AchievementUnlocked"], player_activity_df["CurrencyEarned"], player_activity_df["CurrencySpent"], player_activity_df["QuestCompleted"], player_activity_df["EnemiesDefeated"], player_activity_df["ItemsCollected"], player_activity_df["Deaths"], player_activity_df["DistanceTravelled"], player_activity_df["ChatMessagesSent"], player_activity_df["TeamEventsParticipated"], dim_game_df["sk_id_game"], dim_activity_type_df["sk_id_activity_type"], dim_play_mode_df["sk_id_play_mode"], current_date().alias("insert_date"), current_date().alias("update_date"))

# Create a temporary view for the fact table
fact_player_activity_df.createOrReplaceTempView("temp_fact_player_activity")

# Write the fact table to the target location using SQL merge query
query = ("""
    MERGE INTO dev_catalog.gold.fact_player_activity AS target
    USING temp_fact_player_activity AS source
    ON target.PlayerID = source.PlayerID AND target.SessionID = source.SessionID
       AND target.sk_id_game = source.sk_id_game AND target.sk_id_activity_type = source.sk_id_activity_type
       AND target.sk_id_play_mode = source.sk_id_play_mode 
    WHEN MATCHED THEN
        UPDATE SET 
            PlayerID = source.PlayerID,
            SessionID = source.SessionID,
            sk_id_game = source.sk_id_game,
            sk_id_activity_type = source.sk_id_activity_type,
            sk_id_play_mode = source.sk_id_play_mode,
            Level = source.Level,
            ExperiencePoints = source.ExperiencePoints,
            AchievementUnlocked = source.AchievementUnlocked,
            CurrencyEarned = source.CurrencyEarned,
            CurrencySpent = source.CurrencySpent,
            QuestCompleted = source.QuestCompleted,
            EnemiesDefeated = source.EnemiesDefeated,
            ItemsCollected = source.ItemsCollected,
            Deaths = source.Deaths,
            DistanceTravelled = source.DistanceTravelled,
            ChatMessagesSent = source.ChatMessagesSent,
            TeamEventsParticipated = source.TeamEventsParticipated,
            update_date = source.update_date
            
    WHEN NOT MATCHED THEN
        INSERT (PlayerID, SessionID, Level, ExperiencePoints, AchievementUnlocked, CurrencyEarned, CurrencySpent, QuestCompleted, EnemiesDefeated, ItemsCollected, Deaths, DistanceTravelled, ChatMessagesSent, TeamEventsParticipated, sk_id_game, sk_id_activity_type, sk_id_play_mode, insert_date, update_date)
        VALUES (source.PlayerID, source.SessionID, source.Level, source.ExperiencePoints, source.AchievementUnlocked, source.CurrencyEarned, source.CurrencySpent, source.QuestCompleted, source.EnemiesDefeated, source.ItemsCollected, source.Deaths, source.DistanceTravelled, source.ChatMessagesSent, source.TeamEventsParticipated, source.sk_id_game, source.sk_id_activity_type, source.sk_id_play_mode, source.insert_date, source.update_date)
""")

spark.sql(query)

# Display the fact table
#display(fact_player_activity_df)